In [1]:
import hana_ml
print(f"SAP HANA Client for Python: {hana_ml.__version__}")

### Hana DB connection credentials

import os
file_path = './00-setup/secret.ini' 

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
    from hana_ml.algorithms.pal.utility import Settings
    myhost, myport, myuser, mypwd = Settings.load_config(file_path)
    if myhost == '' or myport == 0 or myuser == '' or mypwd == '':
        print("At least one required connection parameter is empty.")
    else:
        print(f"Connection to SAP HANA db at {myhost}:{myport} \nis established for user {myuser}")
else:
    print(f"The file '{file_path}' does not exist!\nCreate and configure it first.")

#### Connect to Hana DB using hdbcli

from hdbcli import dbapi

# Use connection settings from the environment
connection = dbapi.connect(
    address=myhost,
    port=myport,
    user=myuser,
    password=mypwd,
    autocommit=True,
    sslValidateCertificate=False,
)

print(connection) 

### Connect to Hana DB using hana_ml

from hana_ml import ConnectionContext
# cc = ConnectionContext(userkey='VDB_BETA', encrypt=True)
cc= ConnectionContext(
address=myhost,
port=myport,
user=myuser,
password=mypwd ,
encrypt=True
)
print(cc.hana_version())
print(cc.get_current_schema())

### SAP  AI Core  credentials

import json
import os

# Read configuration from file
config_path = "./.aicore/config.json"
with open(config_path) as config_file:
    config = json.load(config_file)

# Set environment variables
for key, value in config.items():
    os.environ[key] = str(value)

# Print the values to verify
print("AICORE_AUTH_URL:", os.environ.get("AICORE_AUTH_URL"))
print("AICORE_CLIENT_ID:", os.environ.get("AICORE_CLIENT_ID"))
print("AICORE_CLIENT_SECRET:", os.environ.get("AICORE_CLIENT_SECRET"))
print("AICORE_RESOURCE_GROUP:", os.environ.get("AICORE_RESOURCE_GROUP"))
print("AICORE_BASE_URL:", os.environ.get("AICORE_BASE_URL"))


SAP HANA Client for Python: 2.18.23120102
The file './00-setup/secret.ini' exists.
Connection to SAP HANA db at e8cbb57b-f1d9-45df-be0d-ccd5671901ce.hna0.prod-us10.hanacloud.ondemand.com:443 
is established for user developer
<dbapi.Connection Connection object : e8cbb57b-f1d9-45df-be0d-ccd5671901ce.hna0.prod-us10.hanacloud.ondemand.com,443,developer,Hack2Build_Deliotte_7256,True>
4.00.000.00.1708429435 (fa/CE2024.2)
DEVELOPER
AICORE_AUTH_URL: https://deloitte-ci083owd.authentication.eu10.hana.ondemand.com
AICORE_CLIENT_ID: sb-928ec83a-bf7c-465c-aac9-989ed20a8bfb!b410248|aicore!b540
AICORE_CLIENT_SECRET: b76da79b-85c5-4506-b6d0-79cd0eb7df15$TGxTiO6xbkhu-1XY6K8EOGloxqvnDJTZZUlcFhN_piU=
AICORE_RESOURCE_GROUP: default
AICORE_BASE_URL: https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2


In [2]:
### Hana Dataframe

from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser,
    password=mypwd
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")
print(f'Current time on the SAP HANA server: {myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0]}')




Connected to SAP HANA db version 4.00.000.00.1708429435 (fa/CE2024.2) 
at e8cbb57b-f1d9-45df-be0d-ccd5671901ce.hna0.prod-us10.hanacloud.ondemand.com:443 as user developer
Current time on the SAP HANA server: 2024-03-03 15:52:23.966000


In [42]:
### Delete table

# Specify the table name to be deleted
table_name_to_delete = "GL_MANUAL"
cursor = cc.connection.cursor()

# Generate SQL command to drop the table
sql_command = f'DROP TABLE {table_name_to_delete}'

# Execute the SQL command
cursor.execute(sql_command)

# Commit the changes
cc.connection.commit()

# Close the cursor
cursor.close()


In [ ]:

### Create a table

cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE GL_MANUAL(ID BIGINT, SECTION
NVARCHAR(1000), HEADER1 NVARCHAR(1000), HEADER2 NVARCHAR(1000), HEADER3 NVARCHAR(1000), TEXT NVARCHAR(5000)  );'''
cursor.execute(sql_command)
cursor.close()


In [3]:
### import data

import pandas as pd
df = pd.read_csv(r'./00-data/accguide01.csv', sep=';' , low_memory=False)
df.head(3)

,ID,Section,Header1,Header2,Header3,TEXT,CONTENT
0,2.0,Balance Sheet,Intangible assets,Theoretical basis & recognition,NaN,Intangible non-current assets comprise all ass...,Balance SheetIntangible assetsTheoretical basi...
1,NaN,Balance Sheet,Intangible assets,Theoretical basis & recognition,NaN,Acquired intangible assets must be recognized ...,NaN
2,4.0,Balance Sheet,Intangible assets,Theoretical basis & recognition,NaN,COMPANY categorizes the provisions into one of...,Balance SheetIntangible assetsTheoretical basi...


In [4]:
df.shape

(50, 7)

In [45]:
### append records to existing Hana table

from hana_ml.dataframe import create_dataframe_from_pandas
# convert pdf to hdf
v_hdf = create_dataframe_from_pandas(
connection_context=cc,
pandas_df=df,
table_name="GL_MANUAL",
allow_bigint=True,
append=True
)

100%|██████████| 1/1 [00:00<00:00,  3.36it/s]


In [44]:
# alter table -> add fields for mata , text embbeding and  vectors

cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE GL_MANUAL ADD (VEC_META NVARCHAR(5000),VEC_TEXT NCLOB, VEC_VECTOR REAL_VECTOR);'''
cursor.execute(sql_command)
cursor.close()

In [ ]:
!pip install langchain_community
!pip install generative-ai-hub-sdk

In [38]:
# Set the version of the proxy being used
from gen_ai_hub.proxy import set_proxy_version

# Initialize language models from the Generative AI Hub
from gen_ai_hub.proxy.langchain.init_models import init_llm

# Create chat prompt templates for structured input to language models
from langchain.prompts import ChatPromptTemplate

# Parse outputs from OpenAI response
from langchain.schema.output_parser import StrOutputParser

# Parse outputs from OpenAI functions
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Interact with a HANA database for vector storage
from langchain_community.vectorstores import HanaDB

# Initialize OpenAI embeddings for language representation
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')

# Create a Document Store for storing documents
from langchain.docstore.document import Document
# associating Document Store with OpenAI embeddingsa database connection and the source table name 
db = HanaDB(embedding=embeddings, connection=connection, table_name="GL_MANUAL")


In [8]:
#print(db)
#df_transposed = df.T  # Transpose the DataFrame
#df_transposed

In [5]:
### Prerpare emmbedings text 
documents = []
for _, row in df.iterrows():
    # Concatenate Section, Header1, Header2, Header3 with TEXT, using "::" as the separator, with TEXT at the bottom
    content = "::".join([str(row['Section']), str(row['Header1']), str(row['Header2']), str(row['Header3']), str(row['TEXT'])])
    
    # Assuming metadata remains the same
    metadata = {
        "Section": row['Section'],
        "Header1": row['Header1'],
        "Header2": row['Header2'],
        "Header3": row['Header3'],
    }
    
    # Create a Document object with the concatenated content and metadata
    document = [Document(page_content=content, metadata=metadata)]
    documents += document

In [6]:
### create emmbeddings
db.add_documents(documents)

[]

In [21]:
#SSR = db.similarity_search(query , k=4)
#import inspect
#print(dir(SSR))
#print(inspect.getmembers(SSR))
#print(type(SSR))
#help(SSR)

In [7]:
query = "Initial & subsequent measurement"

In [66]:
db.similarity_search(query , k=5)

[Document(page_content='Balance Sheet::Intangible assets::Internally generated intangible assets::Initial and subsequent measurement::Subsequent measurement\nSubsequent measurement of internally generated intangible assets is done at amortized cost in accordance with IAS 38.72.', metadata={'Section': 'Balance Sheet', 'Header1': 'Intangible assets', 'Header2': 'Internally generated intangible assets', 'Header3': 'Initial and subsequent measurement'}),
 Document(page_content='Balance Sheet::Intangible assets::Internally generated intangible assets::Initial and subsequent measurement::Subsequent measurement\nSubsequent measurement of internally generated intangible assets is done at amortized cost in accordance with IAS 38.72.', metadata={'Section': 'Balance Sheet', 'Header1': 'Intangible assets', 'Header2': 'Internally generated intangible assets', 'Header3': 'Initial and subsequent measurement'}),
 Document(page_content='Balance Sheet::Intangible assets::Internally generated intangible 

In [67]:
search_results = db.similarity_search(query , k=2)
print(search_results)

[Document(page_content='Balance Sheet::Intangible assets::Internally generated intangible assets::Initial and subsequent measurement::Subsequent measurement\nSubsequent measurement of internally generated intangible assets is done at amortized cost in accordance with IAS 38.72.', metadata={'Section': 'Balance Sheet', 'Header1': 'Intangible assets', 'Header2': 'Internally generated intangible assets', 'Header3': 'Initial and subsequent measurement'}), Document(page_content='Balance Sheet::Intangible assets::Internally generated intangible assets::Initial and subsequent measurement::Subsequent measurement\nSubsequent measurement of internally generated intangible assets is done at amortized cost in accordance with IAS 38.72.', metadata={'Section': 'Balance Sheet', 'Header1': 'Intangible assets', 'Header2': 'Internally generated intangible assets', 'Header3': 'Initial and subsequent measurement'})]


In [62]:
 set_proxy_version('gen-ai-hub')

llm= init_llm(  
            'gpt-4',
            temperature = 0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            seed=123,
            verbose = False
        )

In [ ]:
for document in search_results:
    prompt = f"Document content: {document.page_content}"
    response = llm.generate(prompt)
    
    # Do something with the generated response, e.g., print it
    print(response)

In [60]:
response = llm.invoke("elaborate on sap HANA Vector database ")
print(response)

content='SAP HANA is an in-memory, column-oriented, relational database management system developed and marketed by SAP SE. Its primary function as a database server is to store and retrieve data as requested by the applications. In addition, it performs advanced analytics (predictive analytics, spatial data processing, text analytics, text search, streaming analytics, graph data processing) and includes ETL capabilities as well as an application server.\n\nThe term "Vector" in SAP HANA Vector database refers to the Vector Engine, which is a part of the SAP HANA database that is designed to speed up data processing. It uses vector processing and parallelization to perform operations on multiple data items simultaneously, rather than one at a time. This can significantly improve the performance of data'


In [56]:
content = f"Document content: {document.page_content}"
# response = llm.generate(prompt)
    
# Do something with the generated response, e.g., print it
print(content)

Document content: Balance Sheet::Intangible assets::Internally generated intangible assets::Initial and subsequent measurement::Subsequent measurement
Subsequent measurement of internally generated intangible assets is done at amortized cost in accordance with IAS 38.72.


In [63]:
prompt = ChatPromptTemplate.from_template("Given the following {content} as SAP S/4HANA Finance module professional, please utilize the provided context to succinctly outline the necessary information for posting transactions in ACDOCA. If the context lacks sufficient details for your task, respond with I need more context anddo not state your role in the response.")
#context = {"content": content} 
chain = (prompt | llm | StrOutputParser())
response = chain.invoke({"content": content})
#generated_text = response['choices'][0]['text']
print(response)




To post transactions in ACDOCA for the subsequent measurement of internally generated intangible assets, you need to follow these steps:

1. Identify the intangible asset: The first step is to identify the internally generated intangible asset that needs to be measured. This could be a patent, trademark, copyright, or any other intangible asset that the company has developed.

2. Calculate the amortized cost: The next step is to calculate the amortized cost of the intangible asset in accordance with IAS 38.72. This involves determining the initial cost of the asset, subtracting any accumulated amortization, and adjusting for any impairment losses.

3. Post the transaction: Once the amortized cost has been calculated, you can post the transaction in ACDOCA. This involves debiting the intangible asset account and crediting the corresponding expense account.

4. Review and verify: After the transaction has been posted, it's important to review and verify the information to ensure that it'

In [ ]:
pip install tkinter

In [3]:
import os
import openai


# Set your OpenAI API credentials
# Set your OpenAI API credentials
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = "https://de-genai-sap.openai.azure.com"  # Your Azure OpenAI resource's endpoint value.
openai.api_key = "1e6adf37c94f4a77bdd34895edc4be61"

# Function to handle the "Ask" button click event
def ask_openai():
    user_question = user_question_entry.get()

    if not user_question.strip():
        # Ensure that the user has entered a question
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, "Please enter a question.")
        return

    # Call OpenAI API to get the assistant's response
    response = openai.ChatCompletion.create(
        engine="GenAITest",
        messages=[
            {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
            {"role": "user", "content": user_question}
        ]
    )

    # Extract and display the assistant's reply
    assistant_reply = response['choices'][0]['message']['content']
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, "Assistant: " + assistant_reply)

#
#